# Home Value Dataset - AAI 540- Module 3


---
Adam Graves
---


Kernel `Python 3 (Data Science)` works well with this notebook.

The following policies need to be attached to the execution role:
- AmazonSageMakerFullAccess
- AmazonS3FullAccess

## Setup SageMaker FeatureStore

Setting up the SageMaker Python SDK and boto client. Note that this notebook requires a `boto3` version above `1.17.21`

In [2]:
#@title 1.1: Loading libraries
import boto3
import sagemaker

original_boto3_version = boto3.__version__
%pip install 'boto3>1.17.21'

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
#@title 1.2: Define session and region for feature store
import sagemaker
from sagemaker import get_execution_role
from sagemaker.session import Session
from sagemaker.feature_store.feature_group import FeatureGroup
from time import gmtime, strftime
import uuid


region = boto3.Session().region_name

boto_session = boto3.Session(region_name=region)

sagemaker_client = boto_session.client(service_name="sagemaker", region_name=region)
featurestore_runtime = boto_session.client(
    service_name="sagemaker-featurestore-runtime", region_name=region
)

feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime,
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


#### S3 Bucket Setup For The OfflineStore

SageMaker FeatureStore writes the data in the OfflineStore of a FeatureGroup to a S3 bucket owned by you. To be able to write to your S3 bucket, SageMaker FeatureStore assumes an IAM role which has access to it. The role is also owned by you.
Note that the same bucket can be re-used across FeatureGroups. Data in the bucket is partitioned by FeatureGroup.

Set the default s3 bucket name and it will be referenced throughout the notebook.

In [4]:
#@title 1.3: Setup S3 bucket
default_s3_bucket_name = feature_store_session.default_bucket()
prefix = "sagemaker-featurestore-m3"

print(default_s3_bucket_name)

sagemaker-us-east-1-995561075051


Set up the IAM role. This role gives SageMaker FeatureStore access to your S3 bucket. 

In [5]:
#@title 1.4: Set IAM role
from sagemaker import get_execution_role

# You can modify the following to use a role of your choosing. See the documentation for how to create this.
role = get_execution_role()
print(role)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
arn:aws:iam::995561075051:role/LabRole


## Inspect Dataset

The provided dataset is a synthetic dataset with two tables: housing gmpas and housing. They can both be joined by the combination of the longitude and latidue column. The gmaps table contains information about location information like city, state, zip code, while the housing has information about the property as in bedrooms, size etc.  

The objective of the model is to query certain perticulars of data on records. 

In [6]:
#@title 2.1: Load datasets from root to my S3 bucket
from botocore.exceptions import NoCredentialsError

def upload_to_s3(local_file, bucket, s3_file):
    """
    Upload a file to an S3 bucket

    :param local_file: File to upload
    :param bucket: Bucket to upload to
    :param s3_file: S3 object name. If not specified then local_file name is used
    :return: True if file was uploaded, else False
    """

    # Create an S3 client
    s3 = boto3.client('s3')
    
    try:
        # Upload the file
        s3.upload_file(local_file, bucket, s3_file)
        print(f"File {local_file} uploaded to {bucket}/{s3_file}")
        return True
    except FileNotFoundError:
        print("The file was not found")
        return False
    except NoCredentialsError:
        print("Credentials not available")
        return False

# Local file paths
local_file_path1 = '/root/aai-540-homework/homework-3-1/housing_gmaps_data_raw.csv'
local_file_path2 = '/root/aai-540-homework/homework-3-1/housing.csv'

# S3 Bucket Name
bucket_name = 'sagemaker-us-east-1-995561075051'

# S3 file paths
s3_file_path1 = 'aai-540-homework/homework-3-1/housing_gmaps_data_raw.csv'
s3_file_path2 = 'aai-540-homework/homework-3-1/housing.csv'

# Upload the files
upload_to_s3(local_file_path1, bucket_name, s3_file_path1)
upload_to_s3(local_file_path2, bucket_name, s3_file_path2)

File /root/aai-540-homework/homework-3-1/housing_gmaps_data_raw.csv uploaded to sagemaker-us-east-1-995561075051/aai-540-homework/homework-3-1/housing_gmaps_data_raw.csv
File /root/aai-540-homework/homework-3-1/housing.csv uploaded to sagemaker-us-east-1-995561075051/aai-540-homework/homework-3-1/housing.csv


True

In [7]:
#@title 2.2: Load datasets to dataframes
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import io

# Initialize S3 client
s3_client = boto3.client("s3", region_name='us-east-1')  

# Specify your S3 bucket and file keys
bucket_name = 'sagemaker-us-east-1-995561075051'
housing_file_key = 'aai-540-homework/homework-3-1/housing.csv'  
gmaps_data_file_key = 'aai-540-homework/homework-3-1/housing_gmaps_data_raw.csv'  

# Load housing data from S3
housing_data_object = s3_client.get_object(Bucket=bucket_name, Key=housing_file_key)
housing_data = pd.read_csv(io.BytesIO(housing_data_object['Body'].read()))

# Load Google Maps data from S3
gmaps_data_object = s3_client.get_object(Bucket=bucket_name, Key=gmaps_data_file_key)
gmaps_data = pd.read_csv(io.BytesIO(gmaps_data_object['Body'].read()))

In [8]:
#@title 2.3: Verify the imported data
housing_data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [9]:
#@title 2.4: Verify the imported data
gmaps_data.head()

,street_number,route,locality-political,administrative_area_level_2-political,administrative_area_level_1-political,country-political,postal_code,address,longitude,latitude,...,establishment-natural_feature,airport-establishment-point_of_interest,political-sublocality-sublocality_level_1,administrative_area_level_3-political,post_box,establishment-light_rail_station-point_of_interest-transit_station,establishment-point_of_interest,aquarium-establishment-park-point_of_interest-tourist_attraction-zoo,campground-establishment-lodging-park-point_of_interest-rv_park-tourist_attraction,cemetery-establishment-park-point_of_interest
0,3130,Grizzly Peak Boulevard,Berkeley,Alameda County,California,United States,94705.0,"3130 Grizzly Peak Blvd, Berkeley, CA 94705, USA",-122.23,37.88,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2005,Tunnel Road,Oakland,Alameda County,California,United States,94611.0,"2005 Tunnel Rd, Oakland, CA 94611, USA",-122.22,37.86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6886,Chabot Road,Oakland,Alameda County,California,United States,94618.0,"6886 Chabot Rd, Oakland, CA 94618, USA",-122.24,37.85,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6365,Florio Street,Oakland,Alameda County,California,United States,94618.0,"6365 Florio St, Oakland, CA 94618, USA",-122.25,37.85,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5407,Bryant Avenue,Oakland,Alameda County,California,United States,94618.0,"5407 Bryant Ave, Oakland, CA 94618, USA",-122.25,37.84,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
#@title 2.5: Set pandas display options to show all columns
pd.set_option('display.max_columns', None)

# Display the entire DataFrame
print(gmaps_data)

      street_number                   route locality-political  \
0              3130  Grizzly Peak Boulevard           Berkeley   
1              2005             Tunnel Road            Oakland   
2              6886             Chabot Road            Oakland   
3              6365           Florio Street            Oakland   
4              5407           Bryant Avenue            Oakland   
...             ...                     ...                ...   
12585           NaN      Garden Valley Road       Camptonville   
12586           NaN           La Porte Road                NaN   
12587         16492      Indiana Ranch Road            Dobbins   
12588         16345             Vierra Road           Rackerby   
12589          9478         Marysville Road       Oregon House   

      administrative_area_level_2-political  \
0                            Alameda County   
1                            Alameda County   
2                            Alameda County   
3                  

# Feature Engineering

In [11]:
#@title 3.1: Check for data issue first on the dataset
# start with housing_data
# Check for NaN values in each column
nan_counts = housing_data.isna().sum()
print("NaN Counts in Each Column:\n", nan_counts)

# Check for empty strings in each column
# This is more relevant for object type columns
empty_string_counts = (housing_data.select_dtypes(include=['object']) == '').sum()
print("\nEmpty String Counts in Each Column:\n", empty_string_counts)

NaN Counts in Each Column:
 longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

Empty String Counts in Each Column:
 ocean_proximity    0
dtype: int64


In [12]:
#@title 3.1.1: Now check gmaps_data
# Check for NaN values in each column
nan_counts = gmaps_data.isna().sum()
print("NaN Counts in Each Column:\n", nan_counts)

# Check for empty strings in each column
# This is more relevant for object type columns
empty_string_counts = (gmaps_data.select_dtypes(include=['object']) == '').sum()
print("\nEmpty String Counts in Each Column:\n", empty_string_counts)

NaN Counts in Each Column:
 street_number                                                                          1402
route                                                                                   380
locality-political                                                                      187
administrative_area_level_2-political                                                    47
administrative_area_level_1-political                                                     3
country-political                                                                         0
postal_code                                                                             180
address                                                                                   0
longitude                                                                                 0
latitude                                                                                  0
neighborhood-political                              

In [13]:
#@title 3.3: Create a unique ID based on the formula -longitude - (longitude - latitude) with a maximum of 3 decimal places
def generate_id(longitude, latitude):
    id_value = -longitude - (longitude - latitude)
    return round(id_value, 3)  # Round to 3 decimal places

housing_data['ID'] = housing_data.apply(lambda row: generate_id(row['longitude'], row['latitude']), axis=1)
gmaps_data['ID'] = gmaps_data.apply(lambda row: generate_id(row['longitude'], row['latitude']), axis=1)

# Check data types
print("Data type of ID in housing_data:", housing_data['ID'].dtype)
print("Data type of ID in gmaps_data:", gmaps_data['ID'].dtype)

# Convert ID columns to string in both DataFrames (make the ID the same)
housing_data['ID'] = housing_data['ID'].astype(str)
gmaps_data['ID'] = gmaps_data['ID'].astype(str)

Data type of ID in housing_data: float64
Data type of ID in gmaps_data: float64


In [14]:
#@title 3.2: # Feature Engineering
#fill empty values with 0
housing_data = housing_data.fillna(0)
gmaps_data = gmaps_data.fillna(0)

# Impute missing values in 'total_bedrooms' with 0 in housing_data
housing_data['total_bedrooms'].fillna(0, inplace=True)

# Impute missing values in 'neighborhood-political' with 'locality-political' in gmaps_data
gmaps_data['neighborhood-political'].fillna(gmaps_data['locality-political'], inplace=True)

# Drop columns with more than 10,000 NaN values
threshold = 10000
gmaps_data.dropna(axis=1, thresh=len(gmaps_data) - threshold, inplace=True)

#One hot encoding for the location
#Hot one encoding and rename fields to lower case
proximity_categories = ['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN']
for category in proximity_categories:
    # Replace '<1H OCEAN' with '1H OCEAN' and create one-hot encoded columns
    column_name = category.lower().replace('<1h ocean', '1h_ocean').replace(' ', '_')
    housing_data[column_name] = (housing_data['ocean_proximity'] == category).astype(int)

# Get the current timestamp
current_timestamp = pd.Timestamp.now()

# Assign the current timestamp to both dataframes
housing_data['event_time'] = current_timestamp
gmaps_data['event_time'] = current_timestamp

In [15]:
#@title 3.3: Calculate the median of 'total_bedrooms' and 'households' for each row and assign it to a new column
# bedrooms_per_household
housing_data['bedrooms_per_household'] = (housing_data['total_bedrooms'] / housing_data['households']).round(6)

In [16]:
#@title 3.4: Set a join between the datasets to group by neighborhood-political and set an average median house value, capped to 500,000

# Step 1: Merge Datasets
merged_data = pd.merge(housing_data, gmaps_data[['ID', 'neighborhood-political']], on='ID')

# Step 2: Group by 'neighborhood-political' and calculate median of 'median_house_value'
neighborhood_medians = merged_data.groupby('neighborhood-political')['median_house_value'].median().reset_index()

# Step 3: Cap the values at 500,000
neighborhood_medians['capped_median_value'] = neighborhood_medians['median_house_value'].apply(lambda x: min(x, 500000))

# Step 4: Join the capped median values back to the original housing_data
final_data = pd.merge(housing_data, neighborhood_medians[['neighborhood-political', 'capped_median_value']], left_on='ID', right_on='neighborhood-political', how='left')

# Rename the capped_median_value to 'average_median_house_value'
final_data.rename(columns={'capped_median_value': 'average_median_house_value'}, inplace=True)

In [17]:
#@title 3.5: Set a join between the datasets to derived from median_house_age average this value across all records for a neighborhood discretized by groups of 10 years i.e. 0-9, 10-19, 20-29
# Step 1: Calculate Median House Age per Neighborhood
median_ages = merged_data.groupby('neighborhood-political')['housing_median_age'].median().reset_index()

# Step 2: Define bin edges
max_age = int(median_ages['housing_median_age'].max()) + 10  
bins = range(0, max_age, 10)

# Adjust the number of labels to match the number of intervals
# Since range does not include the last value, we subtract one interval from max_age for label creation
labels = [f'{i}-{i+9}' for i in range(0, max_age - 10, 10)]

# Step 3: Use pd.cut to discretize
median_ages['age_group'] = pd.cut(median_ages['housing_median_age'], bins=bins, labels=labels, right=False)

In [18]:
#@title 3.6: Set a join between the datasets total households derived from households average this value across all records for a neighborhood must be an integer
# Step 1: Group by 'neighborhood-political' and calculate the average of 'households'
avg_households = merged_data.groupby('neighborhood-political')['households'].mean().reset_index()

# Step 2: Round up the averages to the nearest integer
avg_households['avg_households'] = np.ceil(avg_households['households']).astype(int)

In [19]:
#@title 3.6.1: Define a mapping of long column names to shortened versions (shorten)
column_rename_mapping = {
    'airport-establishment-finance-moving_company-point_of_interest-storage': 'airport_finance_storage',
    'establishment-light_rail_station-point_of_interest-transit_station': 'light_rail_transit_station',
    'aquarium-establishment-park-point_of_interest-tourist_attraction-zoo': 'aquarium_park_zoo',
    'campground-establishment-lodging-park-point_of_interest-rv_park-tourist_attraction': 'campground_lodging_park'
    
}

# Rename the columns in gmaps_data DataFrame
gmaps_data.rename(columns=column_rename_mapping, inplace=True)

# Verify the new column names
print(gmaps_data.columns)

Index(['street_number', 'route', 'locality-political',
       'administrative_area_level_2-political',
       'administrative_area_level_1-political', 'country-political',
       'postal_code', 'address', 'longitude', 'latitude',
       'neighborhood-political', 'postal_code_suffix',
       'establishment-point_of_interest-transit_station',
       'establishment-park-point_of_interest', 'premise',
       'establishment-point_of_interest-subway_station-transit_station',
       'airport_finance_storage', 'subpremise',
       'bus_station-establishment-point_of_interest-transit_station',
       'establishment-park-point_of_interest-tourist_attraction',
       'establishment-natural_feature',
       'airport-establishment-point_of_interest',
       'political-sublocality-sublocality_level_1',
       'administrative_area_level_3-political', 'post_box',
       'light_rail_transit_station', 'establishment-point_of_interest',
       'aquarium_park_zoo', 'campground_lodging_park',
       'cemet

In [20]:
#@title 3.7: Check new columns in dataframes
columns1 = housing_data.columns
print(columns1)

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'ocean_proximity', 'ID', '1h_ocean', 'inland',
       'island', 'near_bay', 'near_ocean', 'event_time',
       'bedrooms_per_household'],
      dtype='object')


## Ingest Data into FeatureStore


In this step we will create the FeatureGroups representing the transaction and identity tables.

#### Define FeatureGroups

In [21]:
#@title 4.1: Define feature groups
from time import gmtime, strftime, sleep

# Define feature group names for housing_data and gmaps_data
housing_feature_group_name = "housing-feature-group-" + strftime("%d-%H-%M-%S", gmtime())
gmaps_feature_group_name = "gmaps-feature-group-" + strftime("%d-%H-%M-%S", gmtime())

In [22]:
#@title 4.2: Build Feature Store
from sagemaker.feature_store.feature_group import FeatureGroup

housing_feature_group = FeatureGroup(
    name=housing_feature_group_name, sagemaker_session=feature_store_session
)
gmaps_feature_group = FeatureGroup(
    name=gmaps_feature_group_name, sagemaker_session=feature_store_session
)

In [23]:
#@title 4.2.1: Convert event_time to Unix timestamp, and define used fields
# Define feature definitions for housing_data
housing_feature_definitions = [
    {"FeatureName": "housing_median_age", "FeatureType": "Fractional"},
    {"FeatureName": "total_rooms", "FeatureType": "Integral"},
    {"FeatureName": "total_bedrooms", "FeatureType": "Integral"},
    {"FeatureName": "population", "FeatureType": "Integral"},
    {"FeatureName": "households", "FeatureType": "Integral"},
    {"FeatureName": "median_house_value", "FeatureType": "Fractional"},
    {"FeatureName": "ocean_proximity", "FeatureType": "Integral"},
    {"FeatureName": "median_income", "FeatureType": "Fractional"},
    {"FeatureName": "1h_ocean", "FeatureType": "Integral"},
    {"FeatureName": "island", "FeatureType": "Integral"},
    {"FeatureName": "inland", "FeatureType": "Integral"},
    {"FeatureName": "near_bay", "FeatureType": "Integral"},
    {"FeatureName": "near_ocean", "FeatureType": "Integral"},
    {"FeatureName": "ID", "FeatureType": "Fractional"},
    {"FeatureName": "bedrooms_per_household", "FeatureType": "Fractional"},
    # Add event_time after defining the list
    {"FeatureName": "event_time", "FeatureType": "Fractional"}
]

# Define feature definitions for gmaps_data
gmaps_feature_definitions = [
    {"FeatureName": "street_number", "FeatureType": "String"},
    {"FeatureName": "locality-political", "FeatureType": "String"},
    {"FeatureName": "country-political", "FeatureType": "String"},
    {"FeatureName": "postal_code", "FeatureType": "Integral"},
    {"FeatureName": "address", "FeatureType": "String"},
    {"FeatureName": "neighborhood-political", "FeatureType": "String"},
    #{"FeatureName": "primary_key_neighborhood", "FeatureType": "String"},
    {"FeatureName": "ID", "FeatureType": "Fractional"},
    # Add event_time after defining the list
    {"FeatureName": "event_time", "FeatureType": "Fractional"}
]

# Convert event_time to Unix timestamp
housing_data['event_time'] = housing_data['event_time'].astype('int64') // 1e9
gmaps_data['event_time'] = gmaps_data['event_time'].astype('int64') // 1e9

# Load data into the feature groups with the updated definitions
housing_feature_group.load_feature_definitions(data_frame=housing_data)
gmaps_feature_group.load_feature_definitions(data_frame=gmaps_data)

[FeatureDefinition(feature_name='street_number', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_name='route', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_name='locality-political', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_name='administrative_area_level_2-political', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_name='administrative_area_level_1-political', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_name='country-political', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_name='postal_code', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='address', feature_type=<FeatureTypeEnum.STRING: 'String'>, collec

In [24]:
#@title 4.3: Generate new, unique feature group names

from time import gmtime, strftime
housing_feature_group_name = "housing-feature-group-" + strftime("%Y%m%d-%H%M%S", gmtime()) + "-" + str(uuid.uuid4())[:8]
gmaps_feature_group_name = "gmaps-feature-group-" + strftime("%Y%m%d-%H%M%S", gmtime()) + "-" + str(uuid.uuid4())[:8]

# Specify the record identifier and event time feature names for housing_data
housing_record_identifier_feature_name = "ID" # The identifier
housing_event_time_feature_name = "event_time"

# Specify the record identifier and event time feature names for gmaps_data
gmaps_record_identifier_feature_name = "ID" # The identifier
gmaps_event_time_feature_name = "event_time"

In [25]:
#@title 4.4: Convert Boolean values, set ID and time fields
import time

current_time_sec = int(round(time.time()))


def cast_object_to_string(data_frame):
    for label in data_frame.columns:
        if data_frame.dtypes[label] == "bool":
            data_frame[label] = data_frame[label].astype("str").astype("string")


# cast object dtype to string. The SageMaker FeatureStore Python SDK will then map the string dtype to String feature type.
cast_object_to_string(housing_data)
cast_object_to_string(gmaps_data)

# record identifier and event time feature names
record_identifier_feature_name = "ID"
event_time_feature_name = "event_time"

# append event_time feature
housing_data[event_time_feature_name] = pd.Series(
    [current_time_sec] * len(housing_data), dtype="float64"
)
gmaps_data[event_time_feature_name] = pd.Series(
    [current_time_sec] * len(gmaps_data), dtype="float64"
)

# load feature definitions to the feature group. SageMaker FeatureStore Python SDK will auto-detect the data schema based on input data.
housing_feature_group.load_feature_definitions(data_frame=housing_data)
# output is suppressed
gmaps_feature_group.load_feature_definitions(data_frame=gmaps_data)
# output is suppressed

[FeatureDefinition(feature_name='street_number', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_name='route', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_name='locality-political', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_name='administrative_area_level_2-political', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_name='administrative_area_level_1-political', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_name='country-political', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_name='postal_code', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='address', feature_type=<FeatureTypeEnum.STRING: 'String'>, collec

#### Create FeatureGroups in SageMaker FeatureStore

In [26]:
#@title 5: Poll the status of the feature group creation
import time

def wait_for_feature_group_creation_complete(feature_group):
    """Wait for the feature group creation to complete by checking the status repeatedly.

    Args:
        feature_group (sagemaker.feature_store.feature_group.FeatureGroup): The feature group to check.
    """
    status = feature_group.describe().get("FeatureGroupStatus")
    print(f"Waiting for feature group {feature_group.name} to be created...")
    while status == "Creating":
        time.sleep(5)
        status = feature_group.describe().get("FeatureGroupStatus")
    if status != "Created":
        raise RuntimeError(f"Failed to create feature group {feature_group.name}")
    print(f"FeatureGroup {feature_group.name} successfully created.")


In [27]:
#@title 5: Create Freature Groups in the Feature store
import sagemaker
from sagemaker import get_execution_role
from sagemaker.session import Session
from sagemaker.feature_store.feature_group import FeatureGroup
from time import gmtime, strftime

# Initialize the SageMaker session
sagemaker_session = Session()

# Get the execution role for the SageMaker session
role = get_execution_role()

# Define your default S3 bucket name and prefix for storing feature groups
default_s3_bucket_name = sagemaker_session.default_bucket()
prefix = 'sagemaker-featurestore-m3'  # Customize this prefix as needed

# The sagemaker_session has already been defined
sagemaker_client = sagemaker_session.sagemaker_client

# Generate feature group names with unique identifiers
housing_feature_group_name = "housing-feature-group-" + strftime("%Y%m%d-%H%M%S", gmtime()) + "-" + str(uuid.uuid4())[:8]
gmaps_feature_group_name = "gmaps-feature-group-" + strftime("%Y%m%d-%H%M%S", gmtime()) + "-" + str(uuid.uuid4())[:8]

# Check if the housing feature group already exists
existing_feature_groups = sagemaker_client.list_feature_groups(NameContains='housing-feature-group')['FeatureGroupSummaries']
existing_names = [fg['FeatureGroupName'] for fg in existing_feature_groups]

if housing_feature_group_name in existing_names:
    print(f"Feature group {housing_feature_group_name} already exists. Consider using a different name or reusing the existing feature group.")
else:
    # Since the feature group doesn't exist, create it
    housing_feature_group = FeatureGroup(name=housing_feature_group_name, sagemaker_session=sagemaker_session)
    housing_feature_group.load_feature_definitions(data_frame=housing_data)  # Assuming housing_data is already defined
    housing_feature_group.create(
        s3_uri=f"s3://{default_s3_bucket_name}/{prefix}",
        record_identifier_name="ID",
        event_time_feature_name="event_time",
        role_arn=role,
        enable_online_store=True
    )
    wait_for_feature_group_creation_complete(housing_feature_group)

# Similarly, check if the gmaps feature group already exists
existing_feature_groups = sagemaker_client.list_feature_groups(NameContains='gmaps-feature-group')['FeatureGroupSummaries']
existing_names = [fg['FeatureGroupName'] for fg in existing_feature_groups]

if gmaps_feature_group_name in existing_names:
    print(f"Feature group {gmaps_feature_group_name} already exists. Consider using a different name or reusing the existing feature group.")
else:
    # Since the feature group doesn't exist, create it
    gmaps_feature_group = FeatureGroup(name=gmaps_feature_group_name, sagemaker_session=sagemaker_session)
    gmaps_feature_group.load_feature_definitions(data_frame=gmaps_data)  # Assuming gmaps_data is already defined
    gmaps_feature_group.create(
        s3_uri=f"s3://{default_s3_bucket_name}/{prefix}",
        record_identifier_name="ID",
        event_time_feature_name="event_time",
        role_arn=role,
        enable_online_store=True
    )
    wait_for_feature_group_creation_complete(gmaps_feature_group)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
Waiting for feature group housing-feature-group-20240129-112005-8bad615d to be created...
FeatureGroup housing-feature-group-20240129-112005-8bad615d successfully created.
Waiting for feature group gmaps-feature-group-20240129-112005-7a506f81 to be created...
FeatureGroup gmaps-feature-group-20240129-112005-7a506f81 successfully created.


In [28]:
### Confirm the FeatureGroup has been created by using the DescribeFeatureGroup and ListFeatureGroups APIs. ###

In [29]:
#@title 5.1: Confirm creation
housing_feature_group.describe()

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:995561075051:feature-group/housing-feature-group-20240129-112005-8bad615d',
 'FeatureGroupName': 'housing-feature-group-20240129-112005-8bad615d',
 'RecordIdentifierFeatureName': 'ID',
 'EventTimeFeatureName': 'event_time',
 'FeatureDefinitions': [{'FeatureName': 'longitude',
   'FeatureType': 'Fractional'},
  {'FeatureName': 'latitude', 'FeatureType': 'Fractional'},
  {'FeatureName': 'housing_median_age', 'FeatureType': 'Fractional'},
  {'FeatureName': 'total_rooms', 'FeatureType': 'Fractional'},
  {'FeatureName': 'total_bedrooms', 'FeatureType': 'Fractional'},
  {'FeatureName': 'population', 'FeatureType': 'Fractional'},
  {'FeatureName': 'households', 'FeatureType': 'Fractional'},
  {'FeatureName': 'median_income', 'FeatureType': 'Fractional'},
  {'FeatureName': 'median_house_value', 'FeatureType': 'Fractional'},
  {'FeatureName': 'ocean_proximity', 'FeatureType': 'String'},
  {'FeatureName': 'ID', 'FeatureType': 'String'},
  {'Featur

In [30]:
#@title 5.1.1: Confirm creation Prting same for next dataset
gmaps_feature_group.describe()

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:995561075051:feature-group/gmaps-feature-group-20240129-112005-7a506f81',
 'FeatureGroupName': 'gmaps-feature-group-20240129-112005-7a506f81',
 'RecordIdentifierFeatureName': 'ID',
 'EventTimeFeatureName': 'event_time',
 'FeatureDefinitions': [{'FeatureName': 'street_number',
   'FeatureType': 'String'},
  {'FeatureName': 'route', 'FeatureType': 'String'},
  {'FeatureName': 'locality-political', 'FeatureType': 'String'},
  {'FeatureName': 'administrative_area_level_2-political',
   'FeatureType': 'String'},
  {'FeatureName': 'administrative_area_level_1-political',
   'FeatureType': 'String'},
  {'FeatureName': 'country-political', 'FeatureType': 'String'},
  {'FeatureName': 'postal_code', 'FeatureType': 'Fractional'},
  {'FeatureName': 'address', 'FeatureType': 'String'},
  {'FeatureName': 'longitude', 'FeatureType': 'Fractional'},
  {'FeatureName': 'latitude', 'FeatureType': 'Fractional'},
  {'FeatureName': 'neighborhood-political', 'F

In [31]:
#@title 5.2: list the feature groups created
sagemaker_client.list_feature_groups()  # use boto client to list FeatureGroups

{'FeatureGroupSummaries': [{'FeatureGroupName': 'neighborhood-feature-group-2024-01-25-12-26-53',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:995561075051:feature-group/neighborhood-feature-group-2024-01-25-12-26-53',
   'CreationTime': datetime.datetime(2024, 1, 25, 12, 28, 30, 676000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created',
   'OfflineStoreStatus': {'Status': 'Active'}},
  {'FeatureGroupName': 'neighborhood-feature-group-2024-01-25-12-01-51',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:995561075051:feature-group/neighborhood-feature-group-2024-01-25-12-01-51',
   'CreationTime': datetime.datetime(2024, 1, 25, 12, 5, 51, 551000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created',
   'OfflineStoreStatus': {'Status': 'Active'}},
  {'FeatureGroupName': 'neighborhood-feature-group-2024-01-25-10-38-05',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:995561075051:feature-group/neighborhood-feature-group-2024-01-25-10-38-05',
   'CreationTime': datetime.da

#### PutRecords into FeatureGroup

After the FeatureGroups have been created, we can put data into the FeatureGroups by using the PutRecord API. This API can handle high TPS and is designed to be called by different streams. The data from all of these Put requests is buffered and written to S3 in chunks. The files will be written to the offline store within a few minutes of ingestion. For this example, to accelerate the ingestion process, we are specifying multiple workers to do the job simultaneously. It will take ~1min to ingest data to the 2 FeatureGroups, respectively.

In [32]:
#@title 6: Ingest the feature groups
housing_feature_group.ingest(data_frame=housing_data, max_workers=3, wait=True)

IngestionManagerPandas(feature_group_name='housing-feature-group-20240129-112005-8bad615d', sagemaker_fs_runtime_client_config=<botocore.config.Config object at 0x7f0976fffc10>, sagemaker_session=<sagemaker.session.Session object at 0x7f09c034cb50>, max_workers=3, max_processes=1, profile_name=None, _async_result=<multiprocess.pool.MapResult object at 0x7f0976fa7bb0>, _processing_pool=<pool ProcessPool(ncpus=1)>, _failed_indices=[])

In [33]:
#@title 6.1: Ingest the feature groups
gmaps_feature_group.ingest(data_frame=gmaps_data, max_workers=3, wait=True)

IngestionManagerPandas(feature_group_name='gmaps-feature-group-20240129-112005-7a506f81', sagemaker_fs_runtime_client_config=<botocore.config.Config object at 0x7f0976fffc10>, sagemaker_session=<sagemaker.session.Session object at 0x7f09c034cb50>, max_workers=3, max_processes=1, profile_name=None, _async_result=<multiprocess.pool.MapResult object at 0x7f097cba5a20>, _processing_pool=<pool ProcessPool(ncpus=1)>, _failed_indices=[])

In [34]:
#@title 6.2: Get a value of a record (ID) to test if exists
# Display the first few rows of the 'ID' column in housing_data
print("First few 'ID' values in housing_data:")
print(housing_data['ID'].head())

# Display the first few rows of the 'ID' column in gmaps_data
print("\nFirst few 'ID' values in gmaps_data:")
print(gmaps_data['ID'].head())

First few 'ID' values in housing_data:
0    282.34
1     282.3
2    282.33
3    282.35
4    282.35
Name: ID, dtype: object

First few 'ID' values in gmaps_data:
0    282.34
1     282.3
2    282.33
3    282.35
4    282.34
Name: ID, dtype: object


In [55]:
#@title 6.3: Check record for first ID found in above list
record_identifier_value = "282.34"  # Ensure this is a string representation of your record ID

# Choose the correct feature group name
feature_group_name = housing_feature_group_name  # or gmaps_feature_group_name, based on where you want to check the record

response = featurestore_runtime.get_record(
    FeatureGroupName=feature_group_name,
    RecordIdentifierValueAsString=record_identifier_value,
)

# Print the retrieved record
print(response['Record'])

[{'FeatureName': 'longitude', 'ValueAsString': '-121.99'}, {'FeatureName': 'latitude', 'ValueAsString': '38.36'}, {'FeatureName': 'housing_median_age', 'ValueAsString': '33.0'}, {'FeatureName': 'total_rooms', 'ValueAsString': '146.0'}, {'FeatureName': 'total_bedrooms', 'ValueAsString': '31.0'}, {'FeatureName': 'population', 'ValueAsString': '75.0'}, {'FeatureName': 'households', 'ValueAsString': '31.0'}, {'FeatureName': 'median_income', 'ValueAsString': '3.5179'}, {'FeatureName': 'median_house_value', 'ValueAsString': '84400.0'}, {'FeatureName': 'ocean_proximity', 'ValueAsString': 'INLAND'}, {'FeatureName': 'ID', 'ValueAsString': '282.34'}, {'FeatureName': '1h_ocean', 'ValueAsString': '0'}, {'FeatureName': 'inland', 'ValueAsString': '1'}, {'FeatureName': 'island', 'ValueAsString': '0'}, {'FeatureName': 'near_bay', 'ValueAsString': '0'}, {'FeatureName': 'near_ocean', 'ValueAsString': '0'}, {'FeatureName': 'event_time', 'ValueAsString': '1706527205.0'}, {'FeatureName': 'bedrooms_per_hous

In [36]:
#@title 6.4: Retrieve full info on both datasets from Feature Groups
response = featurestore_runtime.batch_get_record(
    Identifiers=[
        {
            "FeatureGroupName": housing_feature_group_name,  # Use your housing feature group name
            "RecordIdentifiersValueAsString": ["282.34"],  # Replace with your actual record ID(s) for housing data
        },
        {
            "FeatureGroupName": gmaps_feature_group_name,  # Use your gmaps feature group name
            "RecordIdentifiersValueAsString": ["282.34"],  # Replace with your actual record ID(s) for gmaps data
        },
    ]
)

# Print the retrieved records
for record in response['Records']:
    print(record)


{'FeatureGroupName': 'housing-feature-group-20240129-112005-8bad615d', 'RecordIdentifierValueAsString': '282.34', 'Record': [{'FeatureName': 'longitude', 'ValueAsString': '-121.99'}, {'FeatureName': 'latitude', 'ValueAsString': '38.36'}, {'FeatureName': 'housing_median_age', 'ValueAsString': '33.0'}, {'FeatureName': 'total_rooms', 'ValueAsString': '146.0'}, {'FeatureName': 'total_bedrooms', 'ValueAsString': '31.0'}, {'FeatureName': 'population', 'ValueAsString': '75.0'}, {'FeatureName': 'households', 'ValueAsString': '31.0'}, {'FeatureName': 'median_income', 'ValueAsString': '3.5179'}, {'FeatureName': 'median_house_value', 'ValueAsString': '84400.0'}, {'FeatureName': 'ocean_proximity', 'ValueAsString': 'INLAND'}, {'FeatureName': 'ID', 'ValueAsString': '282.34'}, {'FeatureName': '1h_ocean', 'ValueAsString': '0'}, {'FeatureName': 'inland', 'ValueAsString': '1'}, {'FeatureName': 'island', 'ValueAsString': '0'}, {'FeatureName': 'near_bay', 'ValueAsString': '0'}, {'FeatureName': 'near_ocean

In [37]:
#@title 7: Revisit the filed names and types
print(housing_feature_group.as_hive_ddl())

CREATE EXTERNAL TABLE IF NOT EXISTS sagemaker_featurestore.housing-feature-group-20240129-112005-8bad615d (
  longitude FLOAT
  latitude FLOAT
  housing_median_age FLOAT
  total_rooms FLOAT
  total_bedrooms FLOAT
  population FLOAT
  households FLOAT
  median_income FLOAT
  median_house_value FLOAT
  ocean_proximity STRING
  ID STRING
  1h_ocean INT
  inland INT
  island INT
  near_bay INT
  near_ocean INT
  event_time FLOAT
  bedrooms_per_household FLOAT
  write_time TIMESTAMP
  event_time TIMESTAMP
  is_deleted BOOLEAN
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
  STORED AS
  INPUTFORMAT 'parquet.hive.DeprecatedParquetInputFormat'
  OUTPUTFORMAT 'parquet.hive.DeprecatedParquetOutputFormat'
LOCATION 's3://sagemaker-us-east-1-995561075051/sagemaker-featurestore-m3/995561075051/sagemaker/us-east-1/offline-store/housing-feature-group-20240129-112005-8bad615d-1706527205/data'


In [38]:
#@title 7.1: Revisit the filed names and types (gmaps)
print(gmaps_feature_group.as_hive_ddl())

CREATE EXTERNAL TABLE IF NOT EXISTS sagemaker_featurestore.gmaps-feature-group-20240129-112005-7a506f81 (
  street_number STRING
  route STRING
  locality-political STRING
  administrative_area_level_2-political STRING
  administrative_area_level_1-political STRING
  country-political STRING
  postal_code FLOAT
  address STRING
  longitude FLOAT
  latitude FLOAT
  neighborhood-political STRING
  postal_code_suffix FLOAT
  establishment-point_of_interest-transit_station STRING
  establishment-park-point_of_interest STRING
  premise STRING
  establishment-point_of_interest-subway_station-transit_station STRING
  airport_finance_storage STRING
  subpremise STRING
  bus_station-establishment-point_of_interest-transit_station STRING
  establishment-park-point_of_interest-tourist_attraction STRING
  establishment-natural_feature STRING
  airport-establishment-point_of_interest STRING
  political-sublocality-sublocality_level_1 STRING
  administrative_area_level_3-political STRING
  post_box 

The SageMaker Python SDK’s FeatureStore class also provides the functionality to generate Hive DDL commands. Schema of the table is generated based on the feature definitions. Columns are named after feature name and data-type are inferred based on feature type.

Now let's wait for the data to appear in our offline store before moving forward to creating a dataset. This will take approximately 5 minutes.

In [39]:
#@title 8: Check availabilty of data in the store
account_id = boto3.client("sts").get_caller_identity()["Account"]
print(account_id)

housing_feature_group_resolved_output_s3_uri = (
    housing_feature_group.describe()
    .get("OfflineStoreConfig")
    .get("S3StorageConfig")
    .get("ResolvedOutputS3Uri")
)
gmaps_feature_group_resolved_output_s3_uri = (
    gmaps_feature_group.describe()
    .get("OfflineStoreConfig")
    .get("S3StorageConfig")
    .get("ResolvedOutputS3Uri")
)

housing_feature_group_s3_prefix = housing_feature_group_resolved_output_s3_uri.replace(
    f"s3://{default_s3_bucket_name}/", ""
)
gmaps_feature_group_s3_prefix = gmaps_feature_group_resolved_output_s3_uri.replace(
    f"s3://{default_s3_bucket_name}/", ""
)

offline_store_contents = None
while offline_store_contents is None:
    objects_in_bucket = s3_client.list_objects(
        Bucket=default_s3_bucket_name, Prefix=housing_feature_group_s3_prefix
    )
    if "Contents" in objects_in_bucket and len(objects_in_bucket["Contents"]) > 1:
        offline_store_contents = objects_in_bucket["Contents"]
    else:
        print("Waiting for data in offline store...\n")
        sleep(60)

print("Data available.")

995561075051
Waiting for data in offline store...

Waiting for data in offline store...

Waiting for data in offline store...

Data available.


SageMaker FeatureStore adds metadata for each record that's ingested into the offline store.

## Build Training Dataset

SageMaker FeatureStore automatically builds the Glue Data Catalog for FeatureGroups (you can optionally turn it on/off while creating the FeatureGroup). In this example, we want to create one training dataset with FeatureValues from both identity and transaction FeatureGroups. This is done by utilizing the auto-built Catalog. We run an Athena query that joins the data stored in the offline store in S3 from the 2 FeatureGroups. 

In [40]:
#@title 9: Build training queries for the model
housing_query = housing_feature_group.athena_query()
gmaps_query = gmaps_feature_group.athena_query()

housing_table = housing_query.table_name
gmaps_table = gmaps_query.table_name

# Modify the query string to match your feature groups and join conditions
query_string = (
    f'SELECT * FROM "{housing_table}" '
    f'LEFT JOIN "{gmaps_table}" ON "{housing_table}".ID = "{gmaps_table}".ID'
)
print("Running " + query_string)

# Run Athena query. The output is loaded to a Pandas dataframe.
housing_query.run(
    query_string=query_string,
    output_location=f"s3://{default_s3_bucket_name}/{prefix}/query_results/",
)
housing_query.wait()
dataset = housing_query.as_dataframe()

# Display the first few rows of the dataframe to confirm it loaded correctl
dataset

Running SELECT * FROM "housing_feature_group_20240129_112005_8bad615d_1706527205" LEFT JOIN "gmaps_feature_group_20240129_112005_7a506f81_1706527226" ON "housing_feature_group_20240129_112005_8bad615d_1706527205".ID = "gmaps_feature_group_20240129_112005_7a506f81_1706527226".ID


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,id,1h_ocean,inland,island,near_bay,near_ocean,event_time,bedrooms_per_household,write_time,api_invocation_time,is_deleted,street_number,route,locality-political,administrative_area_level_2-political,administrative_area_level_1-political,country-political,postal_code,address,longitude.1,latitude.1,neighborhood-political,postal_code_suffix,establishment-point_of_interest-transit_station,establishment-park-point_of_interest,premise,establishment-point_of_interest-subway_station-transit_station,airport_finance_storage,subpremise,bus_station-establishment-point_of_interest-transit_station,establishment-park-point_of_interest-tourist_attraction,establishment-natural_feature,airport-establishment-point_of_interest,political-sublocality-sublocality_level_1,administrative_area_level_3-political,post_box,light_rail_transit_station,establishment-point_of_interest,aquarium_park_zoo,campground_lodging_park,cemetery-establishment-park-point_of_interest,id.1,event_time.1,write_time.1,api_invocation_time.1,is_deleted.1
0,-117.41,34.58,14.0,859.0,212.0,541.0,181.0,1.6838,57900.0,INLAND,269.40,0,1,0,0,0,1.706527e+09,1.171271,2024-01-29 11:26:11.939,2024-01-29 11:20:49.000,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-117.41,34.58,10.0,2964.0,668.0,1853.0,609.0,1.6047,73400.0,INLAND,269.40,0,1,0,0,0,1.706527e+09,1.096880,2024-01-29 11:26:11.939,2024-01-29 11:20:49.000,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-117.23,34.49,9.0,4055.0,536.0,1458.0,478.0,5.4201,170600.0,INLAND,268.95,0,1,0,0,0,1.706527e+09,1.121339,2024-01-29 11:26:11.939,2024-01-29 11:20:50.000,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-118.04,33.98,50.0,1951.0,458.0,1362.0,454.0,3.0000,163200.0,<1H OCEAN,270.06,1,0,0,0,0,1.706527e+09,1.008811,2024-01-29 11:26:11.939,2024-01-29 11:20:50.000,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-118.04,33.98,43.0,2446.0,764.0,1699.0,692.0,2.6250,163300.0,<1H OCEAN,270.06,1,0,0,0,0,1.706527e+09,1.104046,2024-01-29 11:26:11.939,2024-01-29 11:20:51.000,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20635,-117.25,34.06,23.0,4503.0,1156.0,3264.0,937.0,1.9821,93000.0,INLAND,268.56,0,1,0,0,0,1.706527e+09,1.233725,2024-01-29 11:26:11.796,2024-01-29 11:22:28.000,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20636,-117.25,34.06,18.0,5009.0,1108.0,2948.0,963.0,3.0042,106500.0,INLAND,268.56,0,1,0,0,0,1.706527e+09,1.150571,2024-01-29 11:26:11.796,2024-01-29 11:22:28.000,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20637,-117.21,34.14,16.0,1613.0,245.0,811.0,267.0,5.2591,140700.0,INLAND,268.56,0,1,0,0,0,1.706527e+09,0.917603,2024-01-29 11:26:11.796,2024-01-29 11:22:28.000,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20638,-117.22,34.12,34.0,2457.0,499.0,1538.0,507.0,2.8090,82500.0,INLAND,268.56,0,1,0,0,0,1.706527e+09,0.984221,2024-01-29 11:26:11.796,2024-01-29 11:22:28.000,False,NaN,NaN,NaN,N

In [41]:
#@title 9.1: Define the columns I want to use in housing_data
selected_columns = [
    "housing_median_age",
    "total_rooms",
    "total_bedrooms",
    "population",
    "households",
    "median_income",
    "ocean_proximity",
    "median_house_value",
    "1h_ocean",
    "inland",
    "island",
    "near_bay",
    "near_ocean",
    "event_time",
    "bedrooms_per_household",
]

In [42]:
#@title 9.2: Prepare query results for training.
query_execution = housing_query.get_query_execution()
query_result = (
    "s3://"
    + default_s3_bucket_name
    + "/"
    + prefix
    + "/query_results/"
    + query_execution["QueryExecution"]["QueryExecutionId"]
    + ".csv"
)
print(query_result)

# Select useful columns for training with target column as the first.
dataset = dataset = dataset[selected_columns]

# Write to csv in S3 without headers and index column.
dataset.to_csv("dataset.csv", header=False, index=False)
s3_client.upload_file("dataset.csv", default_s3_bucket_name, prefix + "/training_input/dataset.csv")
dataset_uri_prefix = "s3://" + default_s3_bucket_name + "/" + prefix + "/training_input/"

dataset

s3://sagemaker-us-east-1-995561075051/sagemaker-featurestore-m3/query_results/7baadaef-4e1a-4cc0-aac5-acae0bbea0bb.csv


,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,median_house_value,1h_ocean,inland,island,near_bay,near_ocean,event_time,bedrooms_per_household
0,14.0,859.0,212.0,541.0,181.0,1.6838,INLAND,57900.0,0,1,0,0,0,1.706527e+09,1.171271
1,10.0,2964.0,668.0,1853.0,609.0,1.6047,INLAND,73400.0,0,1,0,0,0,1.706527e+09,1.096880
2,9.0,4055.0,536.0,1458.0,478.0,5.4201,INLAND,170600.0,0,1,0,0,0,1.706527e+09,1.121339
3,50.0,1951.0,458.0,1362.0,454.0,3.0000,<1H OCEAN,163200.0,1,0,0,0,0,1.706527e+09,1.008811
4,43.0,2446.0,764.0,1699.0,692.0,2.6250,<1H OCEAN,163300.0,1,0,0,0,0,1.706527e+09,1.104046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20635,23.0,4503.0,1156.0,3264.0,937.0,1.9821,INLAND,93000.0,0,1,0,0,0,1.706527e+09,1.233725
20636,18.0,5009.0,1108.0,2948.0,963.0,3.0042,INLAND,106500.0,0,1,0,0,0,1.706527e+09,1.150571
20637,16.0,1613.0,245.0,811.0,267.0,5.2591,INLAND,140700.0,0,1,0,0,0,1.706527e+09,0.917603
20638,34.0,2457.0,499.0,1538.0,507.0,2.8090,INLAND,82500.0,0,1,0,0,0,1.706527e+09,0.984221


In [ ]:
# RUN QUERIES

In [43]:
#@title 10: Query 1 Brooketree
# Existing code to create Athena queries for housing and gmaps feature groups
housing_query = housing_feature_group.athena_query()
gmaps_query = gmaps_feature_group.athena_query()

housing_table = housing_query.table_name
gmaps_table = gmaps_query.table_name

# Query to group records in gmaps_data by ID and filter by "neighborhood-political" = 'Brooktree'
gmaps_subquery = (
    f'SELECT "{gmaps_table}".ID '
    f'FROM "{gmaps_table}" '
    f'WHERE "{gmaps_table}"."neighborhood-political" = \'Brooktree\' '
    f'GROUP BY "{gmaps_table}".ID'
)

# Final query to join the filtered IDs with housing_data and apply additional filters
query_string = (
    f'SELECT "{housing_table}".ID '  
    f'FROM "{housing_table}" '
    f'JOIN ({gmaps_subquery}) gmaps_filtered '
    f'ON "{housing_table}".ID = gmaps_filtered.ID '
    f'WHERE "{housing_table}"."1h_ocean" = 1 '
    f'AND "{housing_table}".inland = 0 '
    f'AND "{housing_table}".island = 0 '
    f'AND "{housing_table}".near_bay = 0 '
    f'AND "{housing_table}".near_ocean = 0 '
    f'AND "{housing_table}".median_house_value = 257400 '
    f'AND "{housing_table}".housing_median_age BETWEEN 0 AND 9 '
    f'AND "{housing_table}".households = 1438 '
    f'AND "{housing_table}".bedrooms_per_household = 1.035334'
)

print("Running " + query_string)

# Run Athena query
housing_query.run(
    query_string=query_string,
    output_location=f"s3://{default_s3_bucket_name}/{prefix}/query_results/",
)
housing_query.wait()
dataset = housing_query.as_dataframe()

# Display the first few rows of the dataframe to confirm it loaded correctly
print(dataset.head())

Running SELECT "housing_feature_group_20240129_112005_8bad615d_1706527205".ID FROM "housing_feature_group_20240129_112005_8bad615d_1706527205" JOIN (SELECT "gmaps_feature_group_20240129_112005_7a506f81_1706527226".ID FROM "gmaps_feature_group_20240129_112005_7a506f81_1706527226" WHERE "gmaps_feature_group_20240129_112005_7a506f81_1706527226"."neighborhood-political" = 'Brooktree' GROUP BY "gmaps_feature_group_20240129_112005_7a506f81_1706527226".ID) gmaps_filtered ON "housing_feature_group_20240129_112005_8bad615d_1706527205".ID = gmaps_filtered.ID WHERE "housing_feature_group_20240129_112005_8bad615d_1706527205"."1h_ocean" = 1 AND "housing_feature_group_20240129_112005_8bad615d_1706527205".inland = 0 AND "housing_feature_group_20240129_112005_8bad615d_1706527205".island = 0 AND "housing_feature_group_20240129_112005_8bad615d_1706527205".near_bay = 0 AND "housing_feature_group_20240129_112005_8bad615d_1706527205".near_ocean = 0 AND "housing_feature_group_20240129_112005_8bad615d_170652

In [45]:
#@title 10.1: Query 1 Brooketree (option b with no bedrooms_per_household = 1.035334) showing that this is the missing value
# Existing code to create Athena queries for housing and gmaps feature groups
housing_query = housing_feature_group.athena_query()
gmaps_query = gmaps_feature_group.athena_query()

housing_table = housing_query.table_name
gmaps_table = gmaps_query.table_name

# Query to group records in gmaps_data by ID and filter by "neighborhood-political" = 'Brooktree'
gmaps_subquery = (
    f'SELECT "{gmaps_table}".ID '
    f'FROM "{gmaps_table}" '
    f'WHERE "{gmaps_table}"."neighborhood-political" = \'Brooktree\' '
    f'GROUP BY "{gmaps_table}".ID'
)

# Final query to join the filtered IDs with housing_data and apply additional filters
query_string = (
    f'SELECT "{housing_table}".ID '  # Replace with the specific columns you need
    f'FROM "{housing_table}" '
    f'JOIN ({gmaps_subquery}) gmaps_filtered '
    f'ON "{housing_table}".ID = gmaps_filtered.ID '
    f'WHERE "{housing_table}"."1h_ocean" = 1 '
    f'AND "{housing_table}".inland = 0 '
    f'AND "{housing_table}".island = 0 '
    f'AND "{housing_table}".near_bay = 0 '
    f'AND "{housing_table}".near_ocean = 0 '
    f'AND "{housing_table}".median_house_value = 257400 '
    f'AND "{housing_table}".housing_median_age BETWEEN 0 AND 9 '
    f'AND "{housing_table}".households = 1438 '
    #f'AND "{housing_table}".bedrooms_per_household = 1.035334'
)

print("Running " + query_string)

# Run Athena query
housing_query.run(
    query_string=query_string,
    output_location=f"s3://{default_s3_bucket_name}/{prefix}/query_results/",
)
housing_query.wait()
dataset = housing_query.as_dataframe()

# Display the first few rows of the dataframe to confirm it loaded correctly
print(dataset.head())

Running SELECT "housing_feature_group_20240129_112005_8bad615d_1706527205".ID FROM "housing_feature_group_20240129_112005_8bad615d_1706527205" JOIN (SELECT "gmaps_feature_group_20240129_112005_7a506f81_1706527226".ID FROM "gmaps_feature_group_20240129_112005_7a506f81_1706527226" WHERE "gmaps_feature_group_20240129_112005_7a506f81_1706527226"."neighborhood-political" = 'Brooktree' GROUP BY "gmaps_feature_group_20240129_112005_7a506f81_1706527226".ID) gmaps_filtered ON "housing_feature_group_20240129_112005_8bad615d_1706527205".ID = gmaps_filtered.ID WHERE "housing_feature_group_20240129_112005_8bad615d_1706527205"."1h_ocean" = 1 AND "housing_feature_group_20240129_112005_8bad615d_1706527205".inland = 0 AND "housing_feature_group_20240129_112005_8bad615d_1706527205".island = 0 AND "housing_feature_group_20240129_112005_8bad615d_1706527205".near_bay = 0 AND "housing_feature_group_20240129_112005_8bad615d_1706527205".near_ocean = 0 AND "housing_feature_group_20240129_112005_8bad615d_170652

In [46]:
#@title 10.3: Query 2 Fisherman’s Wharf 
# Existing code to create Athena queries for housing and gmaps feature groups
housing_query = housing_feature_group.athena_query()
gmaps_query = gmaps_feature_group.athena_query()

housing_table = housing_query.table_name
gmaps_table = gmaps_query.table_name

# Query to group records in gmaps_data by ID and filter by "neighborhood-political" = 'Fisherman’s Wharf'
gmaps_subquery = (
    f'SELECT "{gmaps_table}".ID '
    f'FROM "{gmaps_table}" '
    f'WHERE "{gmaps_table}"."neighborhood-political" = \'Fisherman’s Wharf\' '
    f'GROUP BY "{gmaps_table}".ID'
)

# Final query to join the filtered IDs with housing_data and apply additional filters
query_string = (
    f'SELECT "{housing_table}".ID '  
    f'FROM "{housing_table}" '
    f'JOIN ({gmaps_subquery}) gmaps_filtered '
    f'ON "{housing_table}".ID = gmaps_filtered.ID '
    f'WHERE "{housing_table}"."1h_ocean" = 0 '
    f'AND "{housing_table}".inland = 0 '
    f'AND "{housing_table}".island = 0 '
    f'AND "{housing_table}".near_bay = 1 '
    f'AND "{housing_table}".near_ocean = 0 '
    f'AND "{housing_table}".median_house_value = 500000 '
    f'AND "{housing_table}".housing_median_age BETWEEN 50 AND 59 '
    f'AND "{housing_table}".households = 250 '
    f'AND "{housing_table}".bedrooms_per_household = 1.268'
)

print("Running " + query_string)

# Run Athena query
housing_query.run(
    query_string=query_string,
    output_location=f"s3://{default_s3_bucket_name}/{prefix}/query_results/",
)
housing_query.wait()
dataset = housing_query.as_dataframe()

# Display the first few rows of the dataframe to confirm it loaded correctly
print(dataset.head())

Running SELECT "housing_feature_group_20240129_112005_8bad615d_1706527205".ID FROM "housing_feature_group_20240129_112005_8bad615d_1706527205" JOIN (SELECT "gmaps_feature_group_20240129_112005_7a506f81_1706527226".ID FROM "gmaps_feature_group_20240129_112005_7a506f81_1706527226" WHERE "gmaps_feature_group_20240129_112005_7a506f81_1706527226"."neighborhood-political" = 'Fisherman’s Wharf' GROUP BY "gmaps_feature_group_20240129_112005_7a506f81_1706527226".ID) gmaps_filtered ON "housing_feature_group_20240129_112005_8bad615d_1706527205".ID = gmaps_filtered.ID WHERE "housing_feature_group_20240129_112005_8bad615d_1706527205"."1h_ocean" = 0 AND "housing_feature_group_20240129_112005_8bad615d_1706527205".inland = 0 AND "housing_feature_group_20240129_112005_8bad615d_1706527205".island = 0 AND "housing_feature_group_20240129_112005_8bad615d_1706527205".near_bay = 1 AND "housing_feature_group_20240129_112005_8bad615d_1706527205".near_ocean = 0 AND "housing_feature_group_20240129_112005_8bad615

In [50]:
#Check if Fisherman's Wharf even is in the dataset

distinct_neighborhoods = gmaps_data['neighborhood-political'].unique()

if 'Fisherman’s Wharf' in distinct_neighborhoods:
    print("It is present in gmaps_data.")
else:
    print("It is not present in gmaps_data.")

It is not present in gmaps_data.


In [52]:
#@title 10.4: Query 3 Los Osos 
# Existing code to create Athena queries for housing and gmaps feature groups
housing_query = housing_feature_group.athena_query()
gmaps_query = gmaps_feature_group.athena_query()

housing_table = housing_query.table_name
gmaps_table = gmaps_query.table_name

# Query to group records in gmaps_data by ID and filter by "neighborhood-political" = 'Los Osos'
gmaps_subquery = (
    f'SELECT "{gmaps_table}".ID '
    f'FROM "{gmaps_table}" '
    f'WHERE "{gmaps_table}"."neighborhood-political" = \'Los Osos\' '
    f'GROUP BY "{gmaps_table}".ID'
)

# Final query to join the filtered IDs with housing_data and apply additional filters
query_string = (
    f'SELECT "{housing_table}".ID '  # Replace with the specific columns you need
    f'FROM "{housing_table}" '
    f'JOIN ({gmaps_subquery}) gmaps_filtered '
    f'ON "{housing_table}".ID = gmaps_filtered.ID '
    f'WHERE "{housing_table}"."1h_ocean" = 0 '
    f'AND "{housing_table}".inland = 0 '
    f'AND "{housing_table}".island = 0 '
    f'AND "{housing_table}".near_bay = 0 '
    f'AND "{housing_table}".near_ocean = 1 '
    f'AND "{housing_table}".median_house_value = 221612 '
    f'AND "{housing_table}".housing_median_age BETWEEN 10 AND 19 '
    f'AND "{housing_table}".households = 612 '
    f'AND "{housing_table}".bedrooms_per_household = 1.05'
)

print("Running " + query_string)

# Run Athena query
housing_query.run(
    query_string=query_string,
    output_location=f"s3://{default_s3_bucket_name}/{prefix}/query_results/",
)
housing_query.wait()
dataset = housing_query.as_dataframe()

# Display the first few rows of the dataframe to confirm it loaded correctly
print(dataset.head())

Running SELECT "housing_feature_group_20240129_112005_8bad615d_1706527205".ID FROM "housing_feature_group_20240129_112005_8bad615d_1706527205" JOIN (SELECT "gmaps_feature_group_20240129_112005_7a506f81_1706527226".ID FROM "gmaps_feature_group_20240129_112005_7a506f81_1706527226" WHERE "gmaps_feature_group_20240129_112005_7a506f81_1706527226"."neighborhood-political" = 'Los Osos' GROUP BY "gmaps_feature_group_20240129_112005_7a506f81_1706527226".ID) gmaps_filtered ON "housing_feature_group_20240129_112005_8bad615d_1706527205".ID = gmaps_filtered.ID WHERE "housing_feature_group_20240129_112005_8bad615d_1706527205"."1h_ocean" = 0 AND "housing_feature_group_20240129_112005_8bad615d_1706527205".inland = 0 AND "housing_feature_group_20240129_112005_8bad615d_1706527205".island = 0 AND "housing_feature_group_20240129_112005_8bad615d_1706527205".near_bay = 0 AND "housing_feature_group_20240129_112005_8bad615d_1706527205".near_ocean = 1 AND "housing_feature_group_20240129_112005_8bad615d_1706527

In [53]:
#@title 10.4: Query 3 Los Osos (option b with less filters) showing that this is the missing value
# Existing code to create Athena queries for housing and gmaps feature groups
housing_query = housing_feature_group.athena_query()
gmaps_query = gmaps_feature_group.athena_query()

housing_table = housing_query.table_name
gmaps_table = gmaps_query.table_name

# Query to group records in gmaps_data by ID and filter by "neighborhood-political" = 'Los Osos'
gmaps_subquery = (
    f'SELECT "{gmaps_table}".ID '
    f'FROM "{gmaps_table}" '
    f'WHERE "{gmaps_table}"."neighborhood-political" = \'Los Osos\' '
    f'GROUP BY "{gmaps_table}".ID'
)

# Final query to join the filtered IDs with housing_data and apply additional filters
query_string = (
    f'SELECT "{housing_table}".ID '  # Replace with the specific columns you need
    f'FROM "{housing_table}" '
    f'JOIN ({gmaps_subquery}) gmaps_filtered '
    f'ON "{housing_table}".ID = gmaps_filtered.ID '
    f'WHERE "{housing_table}"."1h_ocean" = 0 '
    f'AND "{housing_table}".inland = 0 '
    f'AND "{housing_table}".island = 0 '
    f'AND "{housing_table}".near_bay = 0 '
    f'AND "{housing_table}".near_ocean = 1 '
    #f'AND "{housing_table}".median_house_value = 221612 '
    #f'AND "{housing_table}".housing_median_age BETWEEN 10 AND 19 '
    #f'AND "{housing_table}".households = 612 '
    #f'AND "{housing_table}".bedrooms_per_household = 1.05'
)

print("Running " + query_string)

# Run Athena query
housing_query.run(
    query_string=query_string,
    output_location=f"s3://{default_s3_bucket_name}/{prefix}/query_results/",
)
housing_query.wait()
dataset = housing_query.as_dataframe()

# Display the first few rows of the dataframe to confirm it loaded correctly
print(dataset.head())

Running SELECT "housing_feature_group_20240129_112005_8bad615d_1706527205".ID FROM "housing_feature_group_20240129_112005_8bad615d_1706527205" JOIN (SELECT "gmaps_feature_group_20240129_112005_7a506f81_1706527226".ID FROM "gmaps_feature_group_20240129_112005_7a506f81_1706527226" WHERE "gmaps_feature_group_20240129_112005_7a506f81_1706527226"."neighborhood-political" = 'Los Osos' GROUP BY "gmaps_feature_group_20240129_112005_7a506f81_1706527226".ID) gmaps_filtered ON "housing_feature_group_20240129_112005_8bad615d_1706527205".ID = gmaps_filtered.ID WHERE "housing_feature_group_20240129_112005_8bad615d_1706527205"."1h_ocean" = 0 AND "housing_feature_group_20240129_112005_8bad615d_1706527205".inland = 0 AND "housing_feature_group_20240129_112005_8bad615d_1706527205".island = 0 AND "housing_feature_group_20240129_112005_8bad615d_1706527205".near_bay = 0 AND "housing_feature_group_20240129_112005_8bad615d_1706527205".near_ocean = 1 
       ID
0  276.98
1  276.98
2  276.95
3  276.96
4  277.0

In [56]:
#@title 10.5: Query 3 Los Osos (option c with present value search) 
# Existing code to create Athena queries for housing and gmaps feature groups
housing_query = housing_feature_group.athena_query()
gmaps_query = gmaps_feature_group.athena_query()

housing_table = housing_query.table_name
gmaps_table = gmaps_query.table_name

# Query to group records in gmaps_data by ID and filter by "neighborhood-political" = 'Los Osos'
gmaps_subquery = (
    f'SELECT "{gmaps_table}".ID '
    f'FROM "{gmaps_table}" '
    f'WHERE "{gmaps_table}"."neighborhood-political" = \'Los Osos\' '
    f'GROUP BY "{gmaps_table}".ID'
)

# Final query to join the filtered IDs with housing_data and apply additional filters
query_string = (
    f'SELECT "{housing_table}".ID '  # Replace with the specific columns you need
    f'FROM "{housing_table}" '
    f'JOIN ({gmaps_subquery}) gmaps_filtered '
    f'ON "{housing_table}".ID = gmaps_filtered.ID '
    f'WHERE "{housing_table}"."1h_ocean" = 0 '
    f'AND "{housing_table}".inland = 0 '
    f'AND "{housing_table}".island = 0 '
    f'AND "{housing_table}".near_bay = 0 '
    f'AND "{housing_table}".near_ocean = 1 '
    f'AND "{housing_table}".housing_median_age BETWEEN 10 AND 19 '
    f'AND "{housing_table}".households = 316 '
    f'AND "{housing_table}".bedrooms_per_household = 1.03481'
)

print("Running " + query_string)

# Run Athena query
housing_query.run(
    query_string=query_string,
    output_location=f"s3://{default_s3_bucket_name}/{prefix}/query_results/",
)
housing_query.wait()
dataset = housing_query.as_dataframe()

# Display the first few rows of the dataframe to confirm it loaded correctly
print(dataset.head())

Running SELECT "housing_feature_group_20240129_112005_8bad615d_1706527205".ID FROM "housing_feature_group_20240129_112005_8bad615d_1706527205" JOIN (SELECT "gmaps_feature_group_20240129_112005_7a506f81_1706527226".ID FROM "gmaps_feature_group_20240129_112005_7a506f81_1706527226" WHERE "gmaps_feature_group_20240129_112005_7a506f81_1706527226"."neighborhood-political" = 'Los Osos' GROUP BY "gmaps_feature_group_20240129_112005_7a506f81_1706527226".ID) gmaps_filtered ON "housing_feature_group_20240129_112005_8bad615d_1706527205".ID = gmaps_filtered.ID WHERE "housing_feature_group_20240129_112005_8bad615d_1706527205"."1h_ocean" = 0 AND "housing_feature_group_20240129_112005_8bad615d_1706527205".inland = 0 AND "housing_feature_group_20240129_112005_8bad615d_1706527205".island = 0 AND "housing_feature_group_20240129_112005_8bad615d_1706527205".near_bay = 0 AND "housing_feature_group_20240129_112005_8bad615d_1706527205".near_ocean = 1 AND "housing_feature_group_20240129_112005_8bad615d_1706527

In [ ]:
#Notes: The first query for Brooktree seems on elemnet was missing to make a match, see ver 2 for a match. 
#For query 2 with Fisherman's Wharf there is no record for that location, see the check. I believe the 
#ercord was removed in the null cleanup, it was a single record
#Query 3 has some records that were for Los Oso but 2 records also cleaned up and removed they might of
#had those values, I made different values to show that the queries work fine. 
# There needs to be a better fine tuning feature engineering to ensure full or close to data record retention, 
# I did many revisions but didn't have enough time to fully explore. 